In [27]:
from cryptography.hazmat.primitives.asymmetric import ec, rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
import os
import time

# Генерація ECC ключів
def generate_ecc_key():
    private_key = ec.generate_private_key(
        ec.SECP256R1(),
        backend=None
    )
    return private_key

# Генерація RSA ключів
def generate_rsa_key():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=None
    )
    return private_key

# Генерація симетричного ключа
def generate_symmetric_key():
    secret_key = os.urandom(32)  # 256 біт
    return secret_key

# Підпис симетричного ключа за допомогою ECC
def sign_symmetric_key_ecc(secret_key, private_key):
    signature = private_key.sign(
        secret_key,
        ec.ECDSA(hashes.SHA256())
    )
    return signature


# Розподіл симетричного ключа за допомогою ECC
def distribute_key_ecc():
    private_key = generate_ecc_key()
    public_key = private_key.public_key()

    # Генеруємо симетричний ключ
    secret_key = generate_symmetric_key()

    # Підписуємо симетричний ключ з використанням приватного ключа ECC
    signature = sign_symmetric_key_ecc(secret_key, private_key)

    # Зашифровуємо секретний ключ разом з підписом, використовуючи ECC
    encrypted_key = secret_key + signature

    return secret_key, encrypted_key

# Розподіл симетричного ключа за допомогою RSA
def distribute_key_rsa():
    private_key = generate_rsa_key()
    public_key = private_key.public_key()

    # Генеруємо симетричний ключ
    secret_key = generate_symmetric_key()

    # Підписуємо симетричний ключ з використанням приватного ключа RSA
    signature = private_key.sign(
        secret_key,
        padding.PKCS1v15(),
        hashes.SHA256()
    )

    # Зашифровуємо секретний ключ разом з підписом, використовуючи RSA
    encrypted_key = secret_key + signature

    return secret_key, encrypted_key

# Порівняння часу розподілу ключів для ECC та RSA
start_time = time.time()
ecc_secret_key, ecc_encrypted_key = distribute_key_ecc()
ecc_time = time.time() - start_time

start_time = time.time()
rsa_secret_key, rsa_encrypted_key = distribute_key_rsa()
rsa_time = time.time() - start_time

print("Час розподілу ключа за допомогою ECC:", ecc_time, "сек")
print("Час розподілу ключа за допомогою RSA:", rsa_time, "сек")
print("Розмір зашифрованого ключа за допомогою ECC:", len(ecc_encrypted_key), "байт")
print("Розмір зашифрованого ключа за допомогою RSA:", len(rsa_encrypted_key), "байт")



Час розподілу ключа за допомогою ECC: 0.0009086132049560547 сек
Час розподілу ключа за допомогою RSA: 0.04292178153991699 сек
Розмір зашифрованого ключа за допомогою ECC: 103 байт
Розмір зашифрованого ключа за допомогою RSA: 288 байт
